# Setup

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt
from PIL import Image
import os
import numpy as np
import yaml
import shutil
import seaborn as sns
%cd /content
%rm -rf LaPeau
!git clone https://github.com/caohieu04/LaPeau
%cd /content/LaPeau/yolov5-master/
!ls
!pip install -r requirements.txt
!nvidia-smi
!pip install wandb
import numpy as np
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
from PIL import Image
import tensorflow as tf
import imghdr
import random

/content
Cloning into 'LaPeau'...
remote: Enumerating objects: 2323, done.
remote: Counting objects: 100% (1312/1312), done.
remote: Compressing objects: 100% (1181/1181), done.
remote: Total 2323 (delta 118), reused 1304 (delta 117), pack-reused 1011
Receiving objects: 100% (2323/2323), 69.50 MiB | 22.81 MiB/s, done.
Resolving deltas: 100% (157/157), done.
/content/LaPeau/yolov5-master
data	      Dockerfile  models	    test.py	    utils
dataset.yaml  hubconf.py  README.md	    train.py	    weights
detect.py     LICENSE	  requirements.txt  tutorial.ipynb
Sun Apr 18 03:30:57 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|       

In [4]:
def f1_score(y_true, y_pred): #taken from old keras source code
    true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
    possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
    predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + K.epsilon())
    recall = true_positives / (possible_positives + K.epsilon())
    f1_val = 2*(precision*recall)/(precision+recall+K.epsilon())
    return f1_val
%cd /content/
!mkdir -p data
!mkdir -p data/train
!mkdir -p data/train/BaKhoang
!mkdir -p data/train/VayNen
!mkdir -p data/train/DaVayCa
!mkdir -p data/train/Zona
!mkdir -p data/val
!mkdir -p data/val/BaKhoang
!mkdir -p data/val/VayNen
!mkdir -p data/val/DaVayCa
!mkdir -p data/val/Zona
import numpy as np
import shutil

/content


# Tune

In [5]:
img_size = 456
y = []
X = []
mp = ['BaKhoang','VayNen','DaVayCa','Zona']
def load_image( infilename ) :
    img = Image.open( infilename )
    img.load()
    data = np.asarray( img, dtype="int32" )
    return data

for t_or_v in ['train', 'val']:
  label_path = '/content/LaPeau/LaPeauData/labels/' + t_or_v
  paths = os.listdir(label_path)

  for p in paths:
    with open(os.path.join(label_path, p), encoding='utf-8-sig') as f:
      tag = int(f.readlines()[0][0])
      src = os.path.join('/content/LaPeau/LaPeauData/images/' + t_or_v, p.split('.')[0] + '.jpg')
      dst = os.path.join('/content/data/' + t_or_v, mp[tag])
      if imghdr.what(src) in ['jpeg', 'png']:
        shutil.copy(src, dst)
      # if not os.path.exists(dst):
      #   shutil.move(src, dst)

def add_noise(img):
    VARIABILITY = 50
    deviation = VARIABILITY*random.random()
    noise = np.random.normal(0, deviation, img.shape)
    img += noise
    np.clip(img, 0., 255.)
    return img
# sample_count = len(os.listdir('/content/data/train'))
batch_size = 16
datagen = tf.keras.preprocessing.image.ImageDataGenerator(
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        horizontal_flip=True,
        vertical_flip=True,
        zoom_range=0.2,
        validation_split=0.2,
        preprocessing_function=add_noise
    )
val_datagen  = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255, validation_split=0.2)

train_loader = datagen.flow_from_directory(
        '/content/data/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode="categorical",
        subset='training'
        )
val_loader = val_datagen.flow_from_directory(
        '/content/data/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode="categorical",
        subset='validation'
        )

from tensorflow.keras.preprocessing.image import ImageDataGenerator as IGD
generators = [IGD(rescale=1./255, validation_split=0.2, rotation_range=20),
             IGD(rescale=1./255, validation_split=0.2, width_shift_range=0.1),
             IGD(rescale=1./255, validation_split=0.2, height_shift_range=0.1),
             IGD(rescale=1./255, validation_split=0.2, horizontal_flip=True),
             IGD(rescale=1./255, validation_split=0.2, vertical_flip=True),
             IGD(rescale=1./255, validation_split=0.2, zoom_range=0.1), 
             IGD(rescale=1./255, validation_split=0.2, preprocessing_function=add_noise) 
             ]

train_x = [[] for _ in range(4)]
train_y = [[] for _ in range(4)]
for i in range(4):
  MP = mp[i]
  for path in os.listdir('/content/data/train/' + MP):
    image_path = os.path.join('/content/data/train/' + MP, path)
    image = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_size, img_size))
    input_arr = keras.preprocessing.image.img_to_array(image)
    # print(input_arr.shape)
    train_x[i].append(input_arr)
    train_y[i].append(i)
  train_x[i] = np.array(train_x[i])
  train_y[i] = np.array(train_y[i])

# print(val_datagen.flow(train_x[0], train_y[0], batch_size=batch_size, subset='training'))

IGD_boo = [[False, False, False, True, True, False, True],
           [False, False, False, True, True, False, True],
           [False, False, False, True, True, False, True],
           [False, False, False, True, True, False, True] 
           ]
# print(IGD_boo) 
batch_size = 4
X = []
Y = []

total_cnt = 0
for id_igd in range(len(generators)):
  igd = generators[id_igd]
  for id in range(4):
    repeat = 5
    for _ in range(repeat):
      if not IGD_boo[id][id_igd]:
        break
      igd_loader = igd.flow(train_x[id], train_y[id], batch_size=1, subset='training')
      X_te = []
      Y_te = []
      cnt = 0
      while cnt < len(train_x[id]):
        batch = igd_loader.next()
        cnt += batch[1].shape[0]
        total_cnt += batch[1].shape[0]
        X_te.append(batch[0])
        Y_te.append(batch[1])
      
      print(len(X))
      if (len(X) == 0):
        X = np.array(X_te)
        Y = np.array(Y_te)
      else:
        X = np.concatenate((X, X_te))
        Y = np.concatenate((Y, Y_te))
print(total_cnt)
X = np.array(X)
Y = np.array(Y)

# while (len(X)) < 40 // batch_size:
#   batch = train_loader.next() 
#   X.append(batch[0])
#   Y.append(batch[1])
# X = np.array(X)
# Y = np.array(Y)
print(X.shape, Y.shape)
# X = X.reshape(X.shape[0] * X.shape[1], X.shape[2], X.shape[3], X.shape[4])
# Y = Y.reshape(Y.shape[0] * Y.shape[1])
# print(X.shape, Y.shape)
batch_size = 16

val_loader = val_datagen.flow_from_directory(
        '/content/data/train',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode="categorical",
        subset='validation'
        )
f
  

# X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.2, random_state=42)
# print(X)
# print(Y)

Found 229 images belonging to 4 classes.
Found 57 images belonging to 4 classes.
0
65
130
195
260
325
400
475
550
625
700
786
872
958
1044
1130
1190
1250
1310
1370
1430
1495
1560
1625
1690
1755
1830
1905
1980
2055
2130
2216
2302
2388
2474
2560
2620
2680
2740
2800
2860
2925
2990
3055
3120
3185
3260
3335
3410
3485
3560
3646
3732
3818
3904
3990
4050
4110
4170
4230
4290
(4290, 1, 456, 456, 3) (4290, 1)
Found 57 images belonging to 4 classes.


<_io.TextIOWrapper name='/content/LaPeau/LaPeauData/labels/val/i156.txt' mode='r' encoding='utf-8-sig'>

In [6]:
from PIL import Image
%cd /content/
!rm -r aguData
!mkdir -p aguData
!mkdir -p aguData/BaKhoang/
!mkdir -p aguData/VayNen
!mkdir -p aguData/DaVayCa
!mkdir -p aguData/Zona
mp = ['BaKhoang','VayNen','DaVayCa','Zona']
%cd aguData

y = Y[0][:]
for id in range(1, len(Y)):
  # print(y.shape, Y[id].shape)
  y = np.concatenate((y, Y[id]))
print(y.shape)
cur_cnt = 0
for id in range(len(X)):
  for batch in range(X[id].shape[0]):
    im = Image.fromarray((X[id][batch] * 255).astype(np.uint8))
    im.save(f'{mp[y[cur_cnt]]}/image_{id}_{batch}.jpeg')
    cur_cnt += 1
  # print(x.shape, X[id].shape)
  # x = np.concatenate((x, X[id]))
print(cur_cnt)
  

%cd ..
# X, Y = x, y

/content
/content/aguData
(4290,)
4290
/content


In [7]:
t1 = len(os.listdir('/content/aguData/BaKhoang'))
t2 = len(os.listdir('/content/aguData/DaVayCa'))
t3 = len(os.listdir('/content/aguData/VayNen'))
t4 = len(os.listdir('/content/aguData/Zona'))
print(t1 + t2 + t3 + t4)

4290


In [8]:
batch_size = 2

train_loader = tf.keras.preprocessing.image.ImageDataGenerator(rescale=1./255).flow_from_directory(
        '/content/aguData/',
        target_size=(img_size, img_size),
        batch_size=batch_size,
        class_mode="categorical",
        subset='training'
        )

Found 4290 images belonging to 4 classes.


# Run

In [ ]:
IMG_SIZE = img_size
IMG_SHAPE = shape = (img_size, img_size, 3)
NUM_CLASSES = 4
base_model = tf.keras.applications.EfficientNetB5(input_shape=IMG_SHAPE, include_top=False, drop_connect_rate=0.4, weights='imagenet')
base_model.trainable = True
batch_size = 4
inputs = tf.keras.Input(IMG_SHAPE)
x = base_model(inputs, training=True)
x = tf.keras.layers.GlobalAveragePooling2D()(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.Dropout(0.5)(x)
#x = tf.keras.layers.Dense(256, activation='relu')(x)
#x = tf.keras.layers.Dense(128, activation='relu')(x)
#x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dense(32, activation='relu')(x)
outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
model = tf.keras.Model(inputs, outputs)
from tensorflow.keras import *
EPOCHS = 300
lr = 0.0001
def scheduler(epoch, lr):
  if epoch < 1:
    return lr
  else:
    return lr * tf.math.exp(-0.05)
#tf.keras.callbacks.LearningRateScheduler(scheduler)
def lr_finder(num_epochs, base_lr=1e-6, max_lr=0.01):
  cur_lr = base_lr
  lrs = []
  accs = []
  best_acc = 0.0
  while cur_lr < max_lr:
    print(f'Training learning rate {cur_lr}')
    callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=40)]
    model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
              loss=tf.keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy', f1_score])
    history = model.fit(train_loader, epochs=EPOCHS, callbacks=[callback], 
              validation_data = val_loader)
    acc = history.history['accuracy'][-1]
    if  acc > best_acc:
      best_acc = acc
      best_lr = lr
    lrs.append(cur_lr)
    accs.append(acc)
    cur_lr *= 3
  print(lrs, accs)
  sns.lineplot(x=lrs, y=accs)
  return best_lr, best_acc

# lr, test_acc = lr_finder(2)


# lr = 0.00007
# warmup_epoch = int(EPOCHS / 20)
# epochs = EPOCHS
# callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=40)]
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
#           loss=tf.keras.losses.CategoricalCrossentropy(),
#           metrics=['accuracy', f1_score])
# history = model.fit(train_loader, epochs=EPOCHS, callbacks=[callback], 
#           validation_data = val_loader)

def unfreeze_model(model):
    for layer in model.layers[-20:]:
        if not isinstance(layer, layers.BatchNormalization):
            layer.trainable = True

    # mcp_save = tf.keras.callbacks.ModelCheckpoint('best_val2.hdf5', save_best_only=True, monitor='val_f1_score', mode='min')
    # optimizer = tf.keras.optimizers.Adam(learning_rate=7e-5)
    # model.compile(optimizer=optimizer, loss="sparse_categorical_crossentropy", metrics=["accuracy", f1_score])

unfreeze_model(model)
epochs = 10  
# param {type: "slider", min:8, max:50}
# callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=40)]
# hist = model.fit(X, Y, epochs=epochs, validation_data=val_loader, callbacks=[callback])

!pip install tensorflow_addons
import sklearn
import tensorflow_addons as tfa
mcp_save = tf.keras.callbacks.ModelCheckpoint('best_val.hdf5', save_best_only=True, monitor='val_f1_score', mode='min')
optimizer = tf.keras.optimizers.Adam(learning_rate=1e-5)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_score])

X_te = X[:]
Y_te = Y[:]
Y_te = tf.keras.utils.to_categorical(Y_te, num_classes=4)
print(Y_te.shape)
epochs=7
history = model.fit(train_loader, epochs=epochs,  validation_data=val_loader, callbacks=[mcp_save])
# history = model.fit(X_te, Y_te, epochs=epochs,  validation_data=val_loader, callbacks=[mcp_save], batch_size=4)

(4290, 4)
Epoch 1/7
1373/2145 [==================>...........] - ETA: 2:23 - loss: 1.7362 - accuracy: 0.3016 - f1_score: 0.2288

In [ ]:
train_loader = datagen.flow_from_directory(
        'data/train',
        target_size=(img_size, img_size),
        batch_size=4,
        class_mode="categorical",
        subset='training'
        )

# After trained

In [ ]:
mcp_save = tf.keras.callbacks.ModelCheckpoint('best_val.hdf5', save_best_only=True, monitor='val_f1_score', mode='min')
optimizer = tf.keras.optimizers.Adam(learning_rate=7e-5)
model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_score])
history = model.fit(X_te, Y_te, epochs=10, validation_data=val_loader, callbacks=[mcp_save], batch_size=4)

In [ ]:
import keras
from matplotlib import pyplot as plt
history = hist
plt.plot(history.history['f1_score'])
print('Average validation F1 Score: ', np.mean(history.history['val_f1_score'][61:]))
print('Average training F1 Score: ', np.mean(history.history['f1_score'][61:]))
plt.plot(history.history['val_f1_score'])
plt.title('model f1_score')
plt.ylabel('f1_score')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

In [ ]:
# num_epochs = 2
# cur_lr = 0.1
# warmup_epoch = int(num_epochs / 10)
# epochs = num_epochs 
# warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=cur_lr,
#                                     total_steps=int(epochs * sample_count / batch_size),
#                                     warmup_learning_rate=0.0,
#                                     warmup_steps=int(warmup_epoch * sample_count / batch_size),
#                                     hold_base_rate_steps=0)
# callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=15), warm_up_lr]
# model.compile(optimizer=tf.keras.optimizers.RMSprop(lr=lr),
#           loss=tf.keras.losses.CategoricalCrossentropy(),
#           metrics=['accuracy'])
# history = model.fit(train_loader, epochs=num_epochs, callbacks=[callback], validation_data=val_loader)
# acc = history.history['accuracy']

In [ ]:
print(val_loader.filenames)

test_ = []
for fn in val_loader.filenames:
  image_path = os.path.join('/content/data/train', fn)
  image = tf.keras.preprocessing.image.load_img(image_path, target_size=(img_size, img_size))
  input_arr = keras.preprocessing.image.img_to_array(image)
  test_.append(input_arr)
test_ = np.array(test_)
print(test_.shape)
# print(test_)


In [ ]:
# from google.colab import files
# model.save('my_model.h5')
files.download('my_model.h5')

In [ ]:
# # print(vars(val_loader))
# def get_npimgs(loader):
#   data_list = []
#   batch_index = 0


#   while batch_index <= loader.batch_index:
#       data = loader.next()
#       data_list.append(data[0])
#       print(len(data), data[0].shape)
#       batch_index = batch_index + 1

#   # now, data_array is the numeric data of whole images
#   data_array = np.array(data_list)
#   print(data_array.shape)
#   return data_array

# y_preded = get_npimgs(val_loader)
# y_preded = np.concatenate(y_preded)

y_pred = model.predict(test_)
y_true = val_loader.classes
# print(y_preded.shape)
# y_pred = model.predict(tf.convert_to_tensor(y_preded)[1])
# print(y_pred)
y_pred = np.argmax(y_pred, axis=1)
f1_ = f1_score(tf.keras.utils.to_categorical(y_pred, num_classes=4), tf.keras.utils.to_categorical(y_true, num_classes=4))
if f1_ < 0.9:
  continue


y_true = np.array(y_true)
y_pred = np.array(y_pred)
print(y_true)
print(y_pred)


from sklearn.metrics import confusion_matrix
# print(confusion_matrix(y_true, y_pred, ))
cmtx = np.round(confusion_matrix(y_true, y_pred, labels=[0, 1, 2, 3]), decimals=2)
print(cmtx)
from matplotlib.pyplot import figure


print(f'f1 score: {f1_}')
figure(figsize=(12, 12), dpi=80)

# sns.heatmap(cmtx, annot=True)
sum = cmtx.sum(axis=1)
cmtx = cmtx / sum[:, ...]
ax = plt.subplot()
sns.heatmap(cmtx, annot=True, fmt='g', ax=ax)
ax.xaxis.set_ticklabels(mp)
ax.yaxis.set_ticklabels(mp)
# print(cmtx)



plt.show()

# tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()
# (tn, fp, fn, tp)

In [ ]:
plt.figure(figsize=(20,20)) # specifying the overall grid size
plt.axis('off')

for i in range(50):
    plt.subplot(5,10,i+1)    # the number of images in the grid is 5*5 (25)
    plt.imshow(X[i])

In [ ]:
# IMG_SHAPE = shape=(300, 300, 3)
# inputs = tf.keras.Input(IMG_SHAPE)
# x = tf.keras.layers.Conv2D(128, 3, padding='same')(inputs)
# x = tf.keras.layers.Conv2D(128, 5, padding='same')(x)
# x = tf.keras.layers.Conv2D(128, 7, padding='same')(x)
# x = tf.keras.layers.GlobalAveragePooling2D()(x)
# # x = tf.keras.layers.Dropout(0.5)(x)
# # x = tf.keras.layers.Dense(128, activation='relu')(x)
# x = tf.keras.layers.Dense(32, activation='relu')(x)
# outputs = tf.keras.layers.Dense(4, activation='softmax')(x)
# model = tf.keras.Model(inputs, outputs)
# EPOCHS = 20
# lr = 0.0001
# warmup_epoch = int(EPOCHS / 20)
# epochs = EPOCHS
# warm_up_lr = WarmUpCosineDecayScheduler(learning_rate_base=lr,
#                                         total_steps=int(epochs * sample_count / batch_size),
#                                         warmup_learning_rate=0.0,
#                                         warmup_steps=int(warmup_epoch * sample_count / batch_size),
#                                         hold_base_rate_steps=0)
# # te_lr = lr
# # lr_np = []
# print(lr)
# # for epoch in range(EPOCHS):
# #   lr_np.append(scheduler(epoch, te_lr))
# #   te_lr = scheduler(epoch, te_lr)
# # sns.set_style("darkgrid")
# # plt.plot(np.array(lr_np))
# # plt.show()

# callback = [tf.keras.callbacks.EarlyStopping(monitor='loss', patience=40), warm_up_lr]
# model.compile(optimizer=tf.keras.optimizers.Adam(lr=lr),
#           loss=tf.keras.losses.CategoricalCrossentropy(),
#           metrics=['accuracy'])
# history = model.fit(train_loader, epochs=EPOCHS, callbacks=[callback], validation_data=val_loader)
# te_lr = lr
# lr_np = []
print(lr)
# for epoch in range(EPOCHS):
#   lr_np.append(scheduler(epoch, te_lr))
#   te_lr = scheduler(epoch, te_lr)
# sns.set_style("darkgrid")
# plt.plot(np.array(lr_np))
# plt.show()

# val_loader = val_datagen.flow_from_directory(
#         'data/val',
#         target_size=(300, 300),
#         batch_size=batch_size,
#         class_mode="categorical",)


# train_loader = tf.keras.preprocessing.image.(
#     '/content/data/train/',
#     labels="inferred",
#     label_mode="int",
#     class_names=None,
#     color_mode="rgb",
#     batch_size=4,
#     image_size=(300, 300),
#     shuffle=True,
#     seed=69,
#     interpolation="bilinear",
# )
# val_loader = tf.keras.preprocessing.image_dataset_from_directory(
#     '/content/data/val/',
#     labels="inferred",
#     label_mode="int",
#     class_names=None,
#     color_mode="rgb",
#     batch_size=4,
#     image_size=(300, 300),
#     shuffle=True,
#     seed=69,
#     interpolation="bilinear",
# )
# train_x = [[] for _ in range(4)]
# train_y = [[] for _ in range(4)]
# for i in range(4):
#   MP = mp[i]
#   for path in os.listdir('/content/data/train/' + MP):
#     image_path = os.path.join('/content/data/train/' + MP, path)
#     image = tf.keras.preprocessing.image.load_img(image_path)
#     input_arr = keras.preprocessing.image.img_to_array(image)
#     # print(input_arr.shape)
#     train_x[i].append(input_arr)
#     train_y[i].append(i)
# print(val_datagen.flow(train_x[0], train_y[0], batch_size=batch_size, subset='training'))
# mcp_save = tf.keras.callbacks.ModelCheckpoint('best_val.hdf5', save_best_only=True, monitor='val_f1_score', mode='min')
# optimizer = tf.keras.optimizers.Adam(learning_rate=7e-5)
# model.compile(optimizer=optimizer, loss="categorical_crossentropy", metrics=["accuracy", f1_score])
# history = model.fit(train_loader, epochs=10, validation_data=val_loader, callbacks=[mcp_save])